### QR code detection

In [5]:
import cv2
import numpy as np

img = cv2.imread('/content/drive/MyDrive/qr_codes_extract/input.jpeg')

# tạo bản copy từ hình ảnh đọc được 
og = img.copy()

# tạo ảnh gray 
gray = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# làm mờ ảnh 
gaussianblur = cv2.GaussianBlur(gray, (7,7), 0)

# phân ngưỡng ảnh gray
otsuthresh = cv2.threshold(gaussianblur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# tạo kernel hình vuông
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

# sử dụng closing transformers 
closed = cv2.morphologyEx(otsuthresh, cv2.MORPH_CLOSE, kernel, iterations=3)

# tìm các đường viền đóng trong ảnh 
contours = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if len(contours) == 2:
    contours = contours[0]
else:
    contours = contours[1]

qrcode = []

approx_list = []

for cnt in contours:
    # tính chu vi của đường viền 
    perim = cv2.arcLength(cnt, True)
    # làm mượt đường viền 
    approx = cv2.approxPolyDP(cnt, 0.05 * perim, True)

    x,y,w,h = cv2.boundingRect(approx)
    area = cv2.contourArea(cnt)
    ratio = w / float(h)

    # kiểm tra xem đường viền có 4 đỉnh và diện tích lớn hơn 1000, tỉ lệ giữa chiều cao và chiều rộng có nhỏ hơn ngưỡng cho trước không (điều kiện hình vuông)
    if len(approx) == 4 and area > 1000 and (ratio > .95 and ratio < 1.05):
        #rec = [area, x, y, w, h] 
        rec = [area, approx]
        approx_list.append(approx)
        # qrcode.append(rec)
        qrcode.append(rec)

# sắp xếp theo chiều giảm dần
qrcode = sorted(qrcode, key=lambda x: x[0], reverse=True)

# lấy ra 3 hình vuông có kích thước gần giống nhau và lớn nhất
rec_list = qrcode[:3]

# tạo 1 list gồm 12 đỉnh 
list_cor = []
for item in rec_list:
    sub_array = item[1].tolist()
    for sub_element in sub_array:
        list_cor.append(sub_element)


# Lấy ra phần tử có giá trị thực lớn nhất
# max_element = max(qrcode, key=lambda x: x[0])

# x, y, w, h = max_element[1:]

# cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)

# cv2_imshow(img)

In [6]:
centroids = []
for square in rec_list:
    x_sum = 0
    y_sum = 0
    for point in square[1]:
      print(point[0][1])
      x_sum += point[0][0]
      y_sum += point[0][1]
    centroid_x = x_sum / len(square[1])
    centroid_y = y_sum / len(square[1])
    centroids.append([centroid_x, centroid_y])

# In kết quả
for i, centroid in enumerate(centroids):
    print("Tọa độ tâm của hình vuông", i+1, "là:", centroid)

In [19]:
# Tìm điểm có tọa độ x nhỏ nhất
min_x_point = min(list_cor, key=lambda p: p[0][0])

# Tìm điểm có tọa độ y nhỏ nhất
min_y_point = min(list_cor, key=lambda p: p[0][1])

# Tìm điểm có tọa độ y lớn nhất
max_y_point = max(list_cor, key=lambda p: p[0][1])

midpoint = [(min_y_point[0][0] + max_y_point[0][0]) // 2, (min_y_point[0][1] + max_y_point[0][1]) // 2]

# tìm điểm còn lại 
max_x_point = symmetric_point = [2 * midpoint[0] - min_x_point[0][0], 2 * midpoint[1] - min_x_point[0][1]]

array([ 2, -1,  1, -1], dtype=int32)

In [17]:
import cv2
import numpy as np

# Đường nối các điểm thành tam giác
points = np.array([min_x_point[0], min_y_point[0], max_x_point, max_y_point[0]], np.int32)

# Đọc hình ảnh
img = cv2.imread("/content/drive/MyDrive/qr_codes_extract/input.jpeg")

# Vẽ tam giác lên hình ảnh
cv2.polylines(img, [points], isClosed=True, color=(0, 255, 0), thickness=10)

# Hiển thị hình ảnh
cv2_imshow(img)

2885

In [ ]:
# Tạo một bản sao của ảnh để vẽ các viền lên đó
contour_image = img.copy()

# Vẽ các viền lên ảnh
cv2.drawContours(contour_image, approx_list, -1, (0, 255, 0), 2)

# Hiển thị ảnh với các viền đã vẽ
cv2_imshow(contour_image)

### Creating heatmap

In [ ]:
# Tính toán tọa độ tâm của hình vuông
center_x = np.mean(points[:, 0])
center_y = np.mean(points[:, 1])

center = (center_x, center_y)

print("Tọa độ tâm của hình vuông:", center)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Đường dẫn tới hình ảnh đầu vào
image_path = "/content/drive/MyDrive/qr_codes_extract/input.jpeg"

# Kích thước của hình vuông
square_size = 400

# Đọc hình ảnh đầu vào
image = cv2.imread(image_path)

# Tạo một ma trận trống có kích thước của hình ảnh
heatmap = np.zeros((image.shape[0], image.shape[1]))

# Tính toán giá trị intensity dựa trên khoảng cách tới tâm hình vuông
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        distance = np.sqrt((i - center_y) ** 2 + (j - center_x) ** 2)
        intensity = np.exp(-distance / square_size)
        heatmap[i, j] = intensity

# Hiển thị biểu đồ heatmap trên hình ảnh
heatmap_rgb = cv2.applyColorMap((heatmap * 255).astype(np.uint8), cv2.COLORMAP_HOT)
green_image = np.full_like(image, (255, 0, 0))
blended_image = cv2.addWeighted(green_image, 0.7, heatmap_rgb, 0.3, 0)

# Hiển thị hình ảnh
plt.imshow(cv2.cvtColor(blended_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()